In [1]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import Aer, execute
from IPython.display import Image
from qiskit.visualization import plot_histogram

import numpy as np

class circuit_common:
    def __init__(self, *op_args):
        self.index_qubit = 0
        self.store_qubit_len = 0
        self.store_qubit = []
        self.out_index_qubit = []
        self.backend = Aer.backends(name='qasm_simulator')[0]

    def __str__(self):
        """output formatting"""
        out_index_no = len(self.out_index_qubit)
        print('output')
        str_out = ""
        if out_index_no > 0:
            loop = 0
            str_out += '1/sqrt('
            str_out += str(out_index_no)
            str_out += ')*(|'
            while loop < (out_index_no -1):
                str_out += str(np.binary_repr(self.out_index_qubit[loop], width=self.index_qubit))
                str_out += '> + |'
                loop += 1
            str_out += str(np.binary_repr(self.out_index_qubit[loop], width=self.index_qubit))
            str_out += '>)'
            return str_out
        else:
            str_out += 'No matched found for the criteria'
            return str_out
        
    def circuit_preparation(self):
        """Preparation of index q-register"""
        self.qr_index = QuantumRegister(self.index_qubit)
        """Preparation of stored q-state against each q-index"""
        self.qr = QuantumRegister(self.store_qubit_len)
        """Preparation of condition register to be operated"""
        self.qr_cond = QuantumRegister(self.store_qubit_len - 1)
        """out condition register to be measured"""
        self.qr_out = QuantumRegister(1)
        """classical registers for the measurement"""
        self.cr = ClassicalRegister(self.index_qubit + 1)
        self.qc = QuantumCircuit(self.qr_index, self.qr, self.qr_cond, self.qr_out, self.cr)        
        
    def circuit_execution(self, index, decimal_val):
        count = 0
        dec_val = decimal_val
        """converting each input decimal value to binary for stored q-state preparation"""
        while decimal_val != 0:
            if (np.bitwise_and(decimal_val, 1)) == 1:
                self.qc.x(self.qr[count])
            count += 1
            decimal_val = np.right_shift(decimal_val, 1)
          
        count = 0
        """main logic"""
        while count < (self.store_qubit_len - 1):
            self.qc.cx(self.qr[count+1], self.qr[count])
            count += 1
            
        count = 0
        """put and then look for all condition registers"""
        while count < (self.store_qubit_len - 1):
            self.qc.cx(self.qr[count], self.qr_cond[count])
            count += 1
        
        """flip out registers so that if condition is met '0' will be the readout"""
        self.qc.mct(self.qr_cond, self.qr_out)
        self.qc.x(self.qr_out)
            
        self.qc.measure(self.qr_out, self.cr[self.index_qubit])
        count = 0
        """if out register is 0 then measure all the index registers"""
        while count < self.index_qubit:
            self.qc.measure(self.qr_index[count], self.cr[count])
            count += 1
                
        job = execute(self.qc, self.backend)
        cur_result = job.result()
        
        outcome = 0
        """store only the index q-registers when the condition is met"""
        for key in cur_result.get_counts():
            outcome = int(key)
            if outcome < np.left_shift(1, self.index_qubit):
                self.out_index_qubit.append(index)
        
        """Sample q-circuit"""
        print('Circuit for index {} with decimal value passed as {}'.format(index, dec_val))
        print('----------------------------------')
        print(self.qc.draw(output='text'))
                    
            
"""This class is fixed for 2 qubits index input with vector of decimal values 1, 5, 7, 10"""
class circuit_fixed(circuit_common):
    def __init__(self, *op_args):
        super().__init__(*op_args)
        self.index_qubit = 2;
        self.store_qubit.append(1)
        self.store_qubit.append(5)
        self.store_qubit.append(7)
        self.store_qubit.append(10)

        loop = 0
        while loop < len(self.store_qubit):
            data = self.store_qubit[loop]
            data_bit_len = 0
            while data != 0:
                data = np.right_shift(data, 1)
                data_bit_len += 1
            if data_bit_len > self.store_qubit_len:
                self.store_qubit_len = data_bit_len
            loop += 1            

    def circuit_output(self):
        index = 0
        """process the output"""
        while index < np.left_shift(1, self.index_qubit):            
            self.circuit_preparation()
            self.circuit_execution(index, self.store_qubit[index])
            index += 1            
        
"""This class is for generic input of 'n' qubits of indices having any 'm' length qubit data"""
"""Total number of 'm' length qubit data will be 2 to the power n (the index qubit)"""
class circuit_generic(circuit_common):
    def __int__(self, *op_args):
        super().__init__(*op_args)
       
    def get_user_input(self):
        print('Enter number of qubits followed by decimal input after each enter')
        self.index_qubit = int(input())
        tmp_array = [int(input()) for qu in range (np.left_shift(1, self.index_qubit))]
        loop = 0
        while loop < len(tmp_array):
            self.store_qubit.append(tmp_array[loop])
            loop += 1
        
        loop = 0
        while loop < len(self.store_qubit):            
            data = self.store_qubit[loop]
            data_bit_len = 0
            while data != 0:
                data = np.right_shift(data, 1)
                data_bit_len += 1
            if data_bit_len > self.store_qubit_len:
                self.store_qubit_len = data_bit_len
            loop += 1            

    def circuit_output(self):
        index = 0
        while index < np.left_shift(1, self.index_qubit):
            self.circuit_preparation()
            self.circuit_execution(index, self.store_qubit[index])
            index += 1

print('Circuit and Output when input is fixed')
print('--------------------------------------')
qcr_fix = circuit_fixed()
qcr_fix.circuit_output()
print(qcr_fix)

print('Circuit and Output For User defined Input')
print('Use:- 1) Number of Index qubits 2) followed by "m" length of 2 to the power n numbers data input')
print('--------------------------------------------')
qcr_user_defined = circuit_generic();
qcr_user_defined.get_user_input()
qcr_user_defined.circuit_output();
print(qcr_user_defined)


Circuit and Output when input is fixed
--------------------------------------
Circuit for index 0 with decimal value passed as 1
----------------------------------
           ┌─┐                                              
q0_0: ─────┤M├──────────────────────────────────────────────
           └╥┘┌─┐                                           
q0_1: ──────╫─┤M├───────────────────────────────────────────
      ┌───┐ ║ └╥┘┌───┐                                      
q1_0: ┤ X ├─╫──╫─┤ X ├───────■──────────────────────────────
      └───┘ ║  ║ └─┬─┘┌───┐  │                              
q1_1: ──────╫──╫───■──┤ X ├──┼─────────■────────────────────
            ║  ║      └─┬─┘  │  ┌───┐  │                    
q1_2: ──────╫──╫────────■────┼──┤ X ├──┼────■───────────────
            ║  ║             │  └─┬─┘  │    │               
q1_3: ──────╫──╫─────────────┼────■────┼────┼───────────────
            ║  ║           ┌─┴─┐       │    │               
q2_0: ──────╫──╫───────────┤ X ├───────┼───